# LOAD virtual environment

In [ ]:
# create virtual environment
!python3.10 -m venv .venv 

In [ ]:
# activate vitual environment
!source .venv/bin/activate

In [ ]:
# install modules required
pip install -r streamlit_requirements.txt

# CREATE images

In [1]:
import cv2
import os
import time 
import uuid 

In [2]:
IMAGES_PATH = "Tensorflow/workspace/images/collectedimages"

In [3]:
# create the labels = sign language glosses the model shall detect
labels = [
    # test glosses from PHOENIX weather file 1May_2010_Saturday_tagesschau_default-11 1 signer05 0.0 1.79769e+308
    "montag", "auch", "mehr", "wolke", "als", "sonne", "ueberwiegend", "regen", "gewitter",
    ]

number_imgs = 15 # suffices for detecting glosses in similar environment, but is not sufficient for robust model 

In [ ]:
# take 15 images per gloss sequentially via webcam
for label in labels:
    os.makedirs("Tensorflow/workspace/images/collectedimages/", exist_ok=True) # create folder collectedimages
    cap = cv2.VideoCapture(0)
    print("collecting images for {}".format(label))
    time.sleep(5) # 5s time lapse before starting collection of glosses
    for imgnum in range (number_imgs):
        ret, frame = cap.read()
        imagename = os.path.join(IMAGES_PATH, f"{label}.{uuid.uuid1()}.jpg")
        cv2.imwrite(imagename, frame)
        cv2.imshow("frame", frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    cap.release()
    cv2.destroyAllWindows()  

In [ ]:
# update canvas.py toe ensure react boxes can be drawn with labelimg
import os

def update_canvas_file(file_path):
    original_lines = {
        526: "p.drawRect(left_top.x(), left_top.y(), rect_width, rect_height)",
        530: "p.drawLine(self.prev_point.x(), 0, self.prev_point.x(), self.pixmap.height())",
        531: "p.drawLine(0, self.prev_point.y(), self.pixmap.width(), self.prev_point.y())"
    }
    replacement_lines = {
        526: "p.drawRect(int(left_top.x()), int(left_top.y()), int(rect_width), int(rect_height))",
        530: "p.drawLine(int(self.prev_point.x()), 0, int(self.prev_point.x()), self.pixmap.height())",
        531: "p.drawLine(0, int(self.prev_point.y()), self.pixmap.width(), int(self.prev_point.y()))"
    }
    
    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Update the lines
    for line_num in sorted(original_lines.keys(), reverse=True):
        original_line = original_lines[line_num]
        if original_line.strip() in lines[line_num - 1].strip():
            indent_level = len(lines[line_num - 1]) - len(lines[line_num - 1].lstrip())
            lines[line_num - 1] = f"# {lines[line_num - 1]}"
            lines.insert(line_num, " " * indent_level + f"{replacement_lines[line_num]}\n")

    # Write the updated lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

# Path to the canvas.py file
file_path = '../.venv/lib/python3.10/site-packages/libs/canvas.py'

# Update the canvas.py file
update_canvas_file(file_path)
print(f"Updated {file_path} successfully.")


Updated ../.venv/lib/python3.10/site-packages/libs/canvas.py successfully.


In [ ]:
# launch labelimg to create bounding boxes and assign classes = sign language gloss to be detected
!labelimg

2025-02-07 21:25:43.895 Python[23038:620452] +[IMKClient subclass]: chose IMKClient_Modern
Traceback (most recent call last):
  File "/Users/maximilianscheel/neuefische/capstone_project/sign2voice_rtod/.venv/lib/python3.10/site-packages/labelImg/labelImg.py", line 965, in scroll_request
    bar.setValue(bar.value() + bar.singleStep() * units)
TypeError: setValue(self, a0: int): argument 1 has unexpected type 'float'


# SPLIT collectedimages dataset into train 80 % & test 20%

In [4]:
# split folder tensorflow/workpace/images/collectedimages into train 80% vs test 20%

import os
import random
import shutil
import math

IMAGES_PATH = "Tensorflow/workspace/images/collectedimages"
TRAIN_PATH = "Tensorflow/workspace/images/train"
TEST_PATH = "Tensorflow/workspace/images/test"

# Ensure train and test directories exist
os.makedirs(TRAIN_PATH, exist_ok=True)
os.makedirs(TEST_PATH, exist_ok=True)

# Get list of all image files
files = [f for f in os.listdir(IMAGES_PATH) if f.endswith('.jpg')]

# Group files by label
label_dict = {}
for file in files:
    label = file.split('.')[0]
    if label not in label_dict:
        label_dict[label] = []
    label_dict[label].append(file)

# Shuffle and split files by label
train_files = []
test_files = []

for label, file_list in label_dict.items():
    random.shuffle(file_list)
    num_train = math.floor(len(file_list) * 0.8)
    train_files.extend(file_list[:num_train])
    test_files.extend(file_list[num_train:])

# Function to copy files to their respective directories
def copy_files(file_list, destination_path):
    for file in file_list:
        base_filename = os.path.splitext(file)[0]
        # Copy image file
        shutil.copy(os.path.join(IMAGES_PATH, file), destination_path)
        # Copy corresponding XML file
        shutil.copy(os.path.join(IMAGES_PATH, base_filename + '.xml'), destination_path)

# Copy train files
copy_files(train_files, TRAIN_PATH)

# Copy test files
copy_files(test_files, TEST_PATH)

print(f"Train and test datasets created. {len(train_files)} images in the train set and {len(test_files)} images in the test set.")

Train and test datasets created. 108 images in the train set and 27 images in the test set.
